In [1]:
%config InlineBackend.figure_format = 'svg' # change output plot display format to 'svg'

# import the required modules for this notebook
import numpy
import matplotlib.pyplot as plt

# import the helper functions from the parent directory,
# these help with things like graph plotting and notebook layout
import sys
sys.path.append('..')
from helper_functions import *

# set things like fonts etc - comes from helper_functions
set_notebook_preferences()

# add a show/hide code button - also from helper_functions
toggle_code(title = "setup code")

## Cambridge UROP 2020: Application of CYCLOPS in identifying rhythms in biological data

_Author: Henry Lim_

### Background

Circadian rhythms influence many aspects of physiology and behavior, and modulate many processes in mammals including body temperature, blood pressure, and locomotor activity. Identifying molecular mechanisms in humans is challenging as existing large-scale datasets rarely include time of day. To address this problem, we combine understanding of periodic structure, evolutionary conservation, and unsupervised machine learning to order unordered human biopsy data along a periodic cycle. This project addresses the problem of inferring time labels from such data to identify circadian rhythms in genes of humans and other mammals.

The algorithm investigated in this project, cyclic ordering by periodic structure (CYCLOPS), utilises evolutionary conservation and machine learning to identify elliptical structure in high-dimensional data. From this structure, CYCLOPS estimates the phase of each sample. We validated CYCLOPS first using artificially-generated oscillatory data followed by temporally ordered mouse and human data and demonstrated its consistency.

### Introduction

The original authors of CYCLOPS implemented the autoencoder structure and conducted downstream analysis in Julia 0.3.10. The associated files are available for download on GitHub (https://github.com/ranafi/CYCLOPS). Much of the first 2-3 weeks was spent on understanding the code in Julia and writing a similar version of CYCLOPS in Python 3.6.10.

### Version info

These are the versions of Python packages used in this project:

_Tensorflow_: 2.3.0 <br>
_Keras_: 2.4.3 <br>
_Scikit-learn_: 0.23.1 <br>
_Pandas_: 1.0.5 <br>
_Matplotlib_: 3.0.0 <br>

In [2]:
import tensorflow, keras, sklearn, pandas, matplotlib
print(tensorflow.__version__)
print(keras.__version__)
print(sklearn.__version__)
print(pandas.__version__)
print(matplotlib.__version__)

2.3.0
2.4.3
0.23.1
1.0.5
3.0.0


#### CYCLOPS v1

The CYCLOPS autoencoder structure was implemented using the __keras__ library, which was chosen for its ability to enable fast experimentation with deep neural networks. The first CYCLOPS model structure created using keras is shown in the diagram below:

<div class ='box'>
<details>
<summary><b> CYCLOPS v1 model diagram </b></summary>

<div>
<img src="img/model_plot_v1.png" width="800"/><br>
<center><figcaption>Fig.1: CYCLOPS_v1 model structure (arbitrary input size)</figcaption></center>
</div>

</details>
</div>

<div class ='box'>
<details> 

<summary><b> Bottleneck layer </b></summary>

<br>

<div>
<img src="img/bottleneck_layer.png" width="750"/><br>
</div>



The bottleneck layer consists of two nodes, initially which are <i>encoder_circular_in_0</i> and <i>encoder_circular_in_1</i>. These are created as Dense layers with one unit. These two bottleneck nodes have to be constrained to lie on a unit circle. In order to achieve this, we divide the value of each node by the square root of the sum of squares of the two nodes (in other words, the magnitude):

<br>

<center>
$a_{0,n} = \frac{a_0}{\sqrt{a_0^2+a_1^2}}$, 
$a_{1,n} = \frac{a_1}{\sqrt{a_0^2+a_1^2}}$
</center>

$a_{0,n}$ and $a_{1,n}$ are the final constrained bottleneck nodes, represented by <i>encoder_circular_out_0</i> and <i>encoder_circular_out_1</i> in the model diagram.


</details>
</div>

Before training is carried out, the neural network starts with some initial weights which are iteratively updated during training. The term kernel_initializer describes the statistical distribution or function to use for initialising these weights. We used a __glorot_normal kernel initializer__, which draws samples from a truncated normal distribution centered on 0 with stddev = $\sqrt\frac{2}{(u_{in} + u_{out})}$, where $u_{in}$ and $u_{out}$ are the number of input and output units in the weight tensor respectively$^{[1]}$.

The CYCLOPS autoencoder network was trained using __Stochastic Gradient Descent (SGD) with momentum__. It differs from regular SGD in the momentum, which is a method that helps accelerate gradients vectors in the right directions, thus resulting in faster convergence$^{[2]}$. In this instance of CYCLOPS, the momentum was set to be 0.5 and the learning rate was set at 0.3. The batch size was set at 10. These values are consistent with those used in the original CYCLOPS paper.

### Artificial Dataset

It was decided that before CYCLOPS could be applied on human/mouse tissue datasets, a good litmus test would be to apply the initial model on an artificially-generated dataset, composed of a mixture of oscillatory and non-oscillatory genes. Testing it on an artificial dataset would allow a good evaluation of the autoencoder model as the actual ordering of the oscillatory genes was known and this could be compared to the result produced by the autoencoder model.  The notebook in which this analysis was done can be found [here](./artificial.ipynb).

The result using 1000 noisy genes, in which 10% are oscillatory, sampled at 48 non-uniform times, is shown below:<br><br>

<div>
<img src="img/artificial_ord.png" width="500"/><br>
<center><figcaption>Fig.2: Plots of CYCLOPS phase against sample collection times for Artificial Data (1000 genes, 48 samples)</figcaption></center>
</div>

<br>
The ordering produced by CYCLOPS as shown in Figure 2 appeared very promising, showing monotonously increasing phase with sample collection times. There is a single phase jump as the phase wraps around $\pi$.

The plots of gene expression against CYCLOPS phase for the 9 genes with the highest $R^2$ are shown below, displaying near-perfect reconstruction.
<br><br>

<div>
<img src="img/artificial.png" width="800"/><br>
<center><figcaption>Fig.3: Plots of gene expression against CYCLOPS phase for the 9 genes with the highest $R^2$ values in the Hughes Liver Dataset</figcaption></center>
</div>

#### CYCLOPS v2

After initial testing on the human and mouse datasets, it was evident that there was plenty of room for improvement in CYCLOPS autoencoder performance. A second version of CYCLOPS was created, and the changes introduced to the original model include:


<li> Added 2 <b>Hidden</b> layers: one between the input and bottleneck layers, and one between the bottleneck and output layers.
<li> Added 4 <b>Dropout</b> layers, in which randomly selected nodes are selected to be 'dropped-out' with a given probability (in this case 20%) each weight update cycle. This has the effect of the network becoming less sensitive to the specific weights of neurons, resulting in a network that is capable of better generalization and is less likely to overfit the training data.
<li> Changed the training loss from MSE to <b>Cosine Similarity</b>. Cosine Similarity was used instead of Mean Squared Error (MSE) because it was deemed as a more suitable loss function in a setting where it is desired to maximise the proximity between the predictions and targets. It is a negative quantity between 0 and -1, where values closer to -1 indicate greater similarity.

<div class ='box'>
<details>
<summary><b> CYCLOPS v2 model diagram </b></summary>

<div>
<img src="img/model_plot_v2.png" width="800"/><br>
<center><figcaption>Fig.4: CYCLOPS_v2 model structure (arbitrary input size)</figcaption></center>
</div>

</details>
</div>

### Dimensionality Reduction <br>

Before feeding the gene expression data into the autoencoder network, it is necessary to project the data onto a lower dimensional space. The two methods of dimensionality reduction explored in this project are __Principal Component Analysis (PCA)__, and a variant of PCA, __Sparse Principal Component Analysis (SPCA)__.

#### PCA

To implement PCA, we used scikit-learn, a software machine learning library available for Python, and compared it to conducting singular value decomposition (SVD) from scratch. SVD states that any matrix A can be factorized as:

$$
A = USV^H
$$


where $U$ and $V$ are orthogonal matrices with orthonormal eigenvectors of $AA^T$ and $A^TA$ respectively. $S$ is a diagonal matrix with singular values, which are the root of the positive eigenvalues of $AA^T$ or $A^TA$ (both have identical positive eigenvalues). The diagonal elements of $S$ are composed of singular values. In their normalised form, the statistical interpretation of singular values is in the form of **variance** in the data explained by the various components. 

We retained singular values (hence principal components) that retained up to __90%__ of the total variance.  We then verified that both methods produced the same principal components (eigengenes).

Initially, it was found that conducting PCA on the entire dataset of genes did not result in good orderings; additionally, the eigengenes produced did not seem to be oscillatory, as we would have expected. To fix this, __variance filtering__ was performed on the data before performing PCA, in which only the genes in the __top 10%__ of variance were retained. This resulted in a much better ordering as well as quicker processing.

#### Sparse PCA (SPCA)

The same library, scikit-learn, was used to implement Sparse PCA. The reason for using Sparse PCA was initially due to poor performance using PCA - this was later improved using variance pre-processing. There are two main parameters: __n_components__, which controls the number of sparse atoms to extract from the data, and __alpha__, which is the sparsity control parameter.

Later, we decided to use [Karthik's Sparse PCA](./spca.py), due to its greater control over both the sparsity control parameter and the number of non-zero sparse components.

### Hughes Liver Dataset

#### PCA

After performing variance filtering and keeping only the genes with the top 10% variance, PCA was performed on the reduced dataset. The three eigengenes with the largest variance produced by PCA are shown in the figure below:

<br>

<div>
<img src="img/h_eig_pca.png" width="500"/><br>
<center><figcaption>Fig.5: Plots of the three eigengenes with the largest variance for the Hughes Liver Dataset produced by PCA</figcaption></center>
</div>

To assess reordering performance, it was decided that a line was to be fit using linear regression to the plot of CYCLOPS phase against sample collection times as shown below in Figure 6. The corresponding $R^2$ value was used to assess the quality of the fit. The full notebook in which this analysis was done can be found [here](./hughes_liver_pca.ipynb).

<br>

<div>
<img src="img/h_pca_ord.png" width="450"/><br>
<center><figcaption>Fig.6: Plot of CYCLOPS phase against sample collection times for the Hughes Liver Dataset (PCA)</figcaption></center>
</div>

<br>
This was repeated 50 times and the $R^2$ value for each pass was recorded. A boxplot was made for the $R^2$ values, as shown below in Figure 7.

<br>

<div>
<img src="img/box_h_pca.png" width="500"/><br>
<center><figcaption>Fig.7: Boxplot of $R^2$ values for the Hughes Liver Dataset</figcaption></center>
</div>

From the boxplot, it is evident that CYCLOPS produces consistently good orderings for the Hughes Liver Dataset, with a median $R^2$ value of 0.987. The full notebook in which this analysis was done can be found [here](./boxplots_pca.ipynb).

#### Sparse PCA

As discussed above, there are two different methods of performing Sparse PCA: scikit-learn and Karthik's method. To compare the reordering performance (as determined by $R^2$ value) for each method, reordering was performed 50 times using each and boxplots of $R^2$ value were made. The Sparse PCA settings used were: 

<li> scikit-learn: n_components = 9, $\alpha$ = 3   &nbsp; &nbsp;   [Notebook](./hughes_liver_spca_scikit.ipynb)
<li> Karthik's: n_components = 9, $\alpha$ = 8     &nbsp; &nbsp;   [Notebook](./hughes_liver_spca_karthik.ipynb)

<br>

<table><tr>
<td> <img src="img/box_h_spca_sk.png" alt="Drawing" style="width: 500px;"/> </td>
<td> <img src="img/box_h_spca_k.png" alt="Drawing" style="width: 510px;"/> </td>
</tr></table>
<center>Fig.8: Comparison of boxplots of $R^2$ values for two types of Sparse PCA: scikit-learn (left) and Karthik's (right) on Hughes Liver Data
</center>

From the above plots, it is evident that there is little difference between the reordering performances of the two Sparse PCA methods. However, Karthik's Sparse PCA was preferred due to its ability to fix the number of non-zero sparse principal components; similar to PCA, it produced consistently good orderings with a good median $R^2$ value of 0.980.

Using Karthik's Sparse PCA, the eigengenes produced were oscillatory, similar to the ones produced by PCA:

<br>

<div>
<img src="img/h_eig_pca.png" width="500"/><br>
<center><figcaption>Fig.9: Plots of the three eigengenes with the largest variance for the Hughes Liver Dataset produced by Sparse PCA (Karthik's Method)</figcaption></center>
</div>

An example of a phase plot produced by CYCLOPS after Sparse PCA, with a line fitted by linear regression, is shown below in Figure 10.

<br>

<div>
<img src="img/h_spca_ord.png" width="700"/><br>
<center><figcaption>Fig.10: Plot of CYCLOPS phase against sample collection times for the Hughes Liver Dataset (Sparse PCA)</figcaption></center>
</div>

### Mouse Liver Dataset

Similar to the Hughes Liver Dataset, CYCLOPS produced very good reorderings for the Mouse Liver Dataset after preprocessing by both PCA and Sparse PCA. The boxplots of $R^2$ value produced using both methods for 50 reorderings illustrate this:

<table><tr>
<td> <img src="img/box_m_pca.png" alt="Drawing" style="width: 450px;"/> </td>
<td> <img src="img/box_m_spca_k.png" alt="Drawing" style="width: 550px;"/> </td>
</tr></table>
<center>Fig.11: Comparison of boxplots of $R^2$ values for PCA and Karthik's Sparse PCA on Mouse Liver Data
</center>

Example of eigengenes produced by Sparse PCA for the Mouse Liver Dataset:

<br>

<div>
<img src="img/m_eig_spca.png" width="450"/><br>
<center><figcaption>Fig.12: Plots of the three eigengenes with the largest variance for the Mouse Liver Dataset produced by Sparse PCA (Karthik's Method)</figcaption></center>
</div>

An example of a phase plot produced by CYCLOPS after Sparse PCA, with a line fitted by linear regression, is shown below in Figure 10.

<br>

<div>
<img src="img/m_spca_ord.png" width="700"/><br>
<center><figcaption>Fig.13: Plot of CYCLOPS phase against sample collection times for the Mouse Liver Dataset (Sparse PCA)</figcaption></center>
</div>

### 12 Tissue Dataset

For the 12 tissue dataset, Karthik's Sparse PCA was the chosen dimensionality reduction method, with settings $\alpha$ = 8 and n_components = 9. Each tissue was fed through CYCLOPS 50 times and the resulting boxplots of $R^2$ values are shown below in Figure 10, ranked by descending order of median $R^2$ value:

<br>

<div>
<img src="img/box_12_tissue.jpg" width="900"/><br>
<center><figcaption>Fig.14: Plots of gene expression against CYCLOPS phase for the 9 genes with the highest $R^2$ values in the Liv_CT Dataset</figcaption></center>
</div>

The six tissues with median $R^2$ value greater than or equal to 0.80 are __Liv, Kid, BFat, Aor, Lun, Cer__. These are deemed to be the tissues that CYCLOPS can reorder reasonably well. There is a significant dropoff in median $R^2$ value after these tissues (to 0.60 for Adr).

The master notebook for analysis on the 12 tissues can be found [here](./12_tissue_master.ipynb). In the notebook, the tissue to be investigated can be changed in cell 3. The notebook on boxplot analysis can be found [here](./12_tissue_boxplots.ipynb).

#### Liv_CT

CYCLOPS demonstrated consistently excellent reordering on Liver data, with a median $R^2$ value of 0.987 and a near-zero interquartile range.

<br>

<div>
<img src="img/liv_gen_r2.png" width="700"/><br>
<center><figcaption>Fig.15: Plot of CYCLOPS phase against sample collection times for the Mouse Liver (Sparse PCA) Dataset</figcaption></center>
</div>

#### Comparison between CYCLOPS_v1 and CYCLOPS_V2

The table below compares the median $R^2$ values for 50 iterations of CYCLOPS_v1 and CYCLOPS_v2 on all the 12 tissue datasets. From this, it is evident that the second version of CYCLOPS gave a better reordering performance as it outperformed the first version in all but three tissues.

<br>

<div>
<img src="img/12_tissue_v1_v2.png" width="500"/><br>
</div>

#### Things tried:

<ol>
<li> Cross-validation to determine the best and worst seeds for kernel initialisations. It was found that the best seed does not necassarily guarantee the best reordering performance.

<br><br>

4-fold cross-validation was conducted on Lun_CT using 40 different seeds, and the average train and test errors were plotted for each as shown below:

<br><br>

<div>
<img src="img/cross_validation_lun.png" width="600"/><br>
<center><figcaption>Fig.16: Average train and test errors for 4-fold cross-validation on 40 different seeds</figcaption></center>
</div>

</ol>

Next, the best and worst seeds were identified and used as the kernel initialisations to perform reordering.

<br>

<table><tr>
<td> <img src="img/lun_best_test.png" alt="Drawing" style="width: 450px;"/> </td>
<td> <img src="img/lun_worst_test.png" alt="Drawing" style="width: 450px;"/> </td>
</tr></table>
<center>Fig.17: Comparison of reordering performance using the best (left) and worst (right) seeds
</center>

From Figure 17, it is clear that the best seed gave a worse reordering than the worst seed, but this could also be a result of the stochastic nature of the autoencoder network.

#### Things to try:
<ol>
<li> Experiment with different values for:
        <ul>
        <li>momentum
        <li>learning rate
        <li>number of epochs
        </ul>
<br>

<li> Experiment with kernel initialisations from a range of statistical distributions
</ol>

### References

[1] Keras, https://keras.rstudio.com/reference/initializer_glorot_normal.html <br>

[2] Towards Data Science (2017), _Stochastic Gradient Descent with momentum_, Available at: https://towardsdatascience.com/stochastic-gradient-descent-with-momentum-a84097641a5d (accessed September 4, 2020)